install Important Library

In [ ]:
pip install beautifulsoup4 requests pandas openpyxl

Importing Important Library

In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

# AirAsia Reviews Scraper

This Python script scrapes passenger reviews from AirAsia Malaysia's page on airlineratings.com.

## Imports



## Main Function

### `scrape_airasia_reviews(max_pages=5)`

This function is responsible for scraping reviews from multiple pages of the AirAsia Malaysia reviews section.



In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_airasia_reviews(max_pages=5):
    base_url = 'https://www.airlineratings.com/passenger-reviews/airasia-malaysia/'
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    all_reviews = []

    for page in range(1, max_pages + 1):
        url = f"{base_url}?page={page}"
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        reviews_container = soup.select_one('.td-post-content')
        if reviews_container:
            reviews = reviews_container.select('.passenger_review')
            for review in reviews:
                customer_name = review.select_one('h3').text.strip() if review.select_one('h3') else "Unknown"

                flight_details = [detail.text.strip() for detail in review.select('.cabin_flown ~ p')]

                flight_type = ', '.join(flight_details[:-1])  # Join all except the last one

                location_date = review.select_one('.review_date').text.strip() if review.select_one('.review_date') else "Unknown"

                overall_score = review.select_one('.ten_star_rating.rating').text if review.select_one('.ten_star_rating.rating') else "N/A"

                rating_h4s = review.select('.passenger_rating_overall h4')
                rating_texts = [h4.text.strip() for h4 in rating_h4s]

                review_text = review.select_one('.passenger_review_body').text.strip() if review.select_one('.passenger_review_body') else "No review text available"

                all_reviews.append({
                    "Customer Name": customer_name,
                    "Flight Type": flight_type,
                    "Location/Date": location_date,
                    "Overall Score": overall_score,
                    "Rating Categories": rating_texts,
                    "Review Text": review_text
                })

                print(f"Extracted Review Details: {customer_name}, {flight_type}, {location_date}, {overall_score}, {rating_texts}, {review_text}")

        next_button = soup.select_one('.ar_button')
        if not next_button:
            print("No next button found, moving to the next iteration.")
        else:
            print("Next button found, continuing to the next page.")

        print("-" * 50)

    df = pd.DataFrame(all_reviews)
    df.to_excel('airasia_reviews_details.xlsx', index=False)

if __name__ == "__main__":
    scrape_airasia_reviews()